In [1]:
from gurobipy import *
import pandas as pd

## Facility Location 


In [2]:
basic_infor = pd.read_excel('IP_Dataset.xlsx','Basic information')
cities = range(len(basic_infor['City']))
markets = range(len(basic_infor['Market']))

city_infor = pd.read_excel('IP_Dataset.xlsx', 'City\'s information')

op_cost = city_infor['Operating cost']
capacity = city_infor['Capacity']

market_infor = pd.read_excel('IP_Dataset.xlsx', 'Market\'s information')

demand = market_infor['Demand']

ship_infor = pd.read_excel('IP_Dataset.xlsx', 'Shipping cost', index_col = 0)
shipping_cost = []

for i in ship_infor.index:
    shipping_cost.append(list(ship_infor.loc[i]))

In [5]:
eg2 = Model('eg2')

# Define variable 

x = []
for j in cities:
    x.append(eg2.addVar(lb = 0, vtype = GRB.BINARY, name = 'x' + str(j + 1)))
y = []
for i in markets:
    y.append([])
    for j in cities:
        y[i].append(eg2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'y' + str(i+1) + str(j+1)))

# Setting the objective function 
eg2.setObjective(
    quicksum(x[j] * op_cost[j] for j in cities) + quicksum(quicksum(y[i][j] * shipping_cost[i][j] for j in cities) for i in markets), GRB.MINIMIZE)

# Add constraints

eg2.addConstrs((quicksum(y[i][j] for i in markets) <= capacity[j] * x[j] for j in cities), 'capacity constraints')
eg2.addConstrs((quicksum(y[i][j] for j in cities) >= demand[i] for i in markets), 'demand constraints')

eg2.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 10 rows, 30 columns and 55 nonzeros
Model fingerprint: 0x263201cb
Variable types: 25 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [2e+00, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+03, 2e+04]
Presolve time: 0.01s
Presolved: 10 rows, 30 columns, 55 nonzeros
Variable types: 25 continuous, 5 integer (5 binary)
Found heuristic solution: objective 313600.00000

Root relaxation: objective 2.528000e+05, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 252800.000    0    5 313600.000 252800.000  19.4%     -    0s
H    0     0                    280400.00000 252800.000  9.84%     -    0s
H    0     

In [31]:
print('Result:')

for j in cities:
    print(x[j].varName, '=', x[j].x)

print('\tMarket1\tMarket2\tMarket3\tMarket4\tMarket5')

for j in cities:
    print('\nCity' + str(j+1), end = " ")
    for i in markets:
        if len(str(y[i][j].x)) < 7:
            print(y[i][j].x, "\t", end = "")
        else:
            print(y[i][j].x, "", end = "")
print('\nz* =', eg2.objVal)

Result:
x1 = 0.0
x2 = 1.0
x3 = 0.0
x4 = 1.0
x5 = 1.0
	Market1	Market2	Market3	Market4	Market5

City1 0.0 	0.0 	0.0 	0.0 	0.0 	
City2 8000.0 	12000.0 0.0 	0.0 	0.0 	
City3 0.0 	0.0 	0.0 	0.0 	0.0 	
City4 0.0 	0.0 	8000.0 	0.0 	17000.0 
City5 0.0 	0.0 	1000.0 	14000.0 0.0 	
z* = 268950.0
